# Hotellibroneering prioritiseeritud liikme vahendustarkvaraga

See märkmik demonstreerib **funktsioonipõhist vahendustarkvara** kasutades Microsoft Agent Frameworki. Me täiendame tingimusliku töövoo näidet, lisades vahendustarkvara kihi, mis annab prioriteetsetele liikmetele erisoodustusi.

## Mida sa õpid:
1. **Funktsioonipõhine vahendustarkvara**: Funktsioonide tulemuste kinni püüdmine ja muutmine
2. **Konteksti ligipääs**: `context.result` lugemine ja muutmine pärast täitmist
3. **Ärilogiika rakendamine**: Prioriteetse liikme hüved
4. **Tulemus Override**: Funktsiooni väljundite muutmine kasutaja staatuse põhjal
5. **Sama töövoog, erinevad tulemused**: Vahendustarkvara juhitud käitumise muudatused

## Töövoo arhitektuur vahendustarkvaraga:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## Peamine erinevus tingimuslikust töövoost:

**Ilma vahendustarkvarata** (14-conditional-workflow.ipynb):
- Pariisil pole tube → Suunamine alternative_agent'i

**Vahendustarkvaraga** (see märkmik):
- Tavaline kasutaja + Pariis → Tube ei ole → Suunamine alternative_agent'i
- Prioriteetne kasutaja + Pariis → 🌟 Vahendustarkvara muudab tulemust! → Saadaval → Suunamine booking_agent'ile

## Eeldused:
- Microsoft Agent Frameworki paigaldus
- Tingimuslike töövoogude mõistmine (vt 14-conditional-workflow.ipynb)
- GitHubi token või OpenAI API võti
- Põhilised teadmised vahendustarkvara mustritest


In [1]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Samm 1: Määratle Pydantic mudelid struktureeritud väljundite jaoks

Need mudelid määratlevad **skeemi**, mida agendid tagastavad. Oleme lisanud välja `priority_override`, et jälgida, millal vahendsõnumid muudavad kättesaadavuse tulemust.


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Step 2: Määra prioriteetsete liikmete andmebaas

Selle demo jaoks simuleerime prioriteetse liikmestiku andmebaasi. Tootmiskeskkonnas küsitaks see päring päris andmebaasist või API-st.

**Prioriteetsed liikmed:**
- `alice@example.com` - VIP liige
- `bob@example.com` - Premium liige  
- `priority_user` - Testkonto


In [3]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## 3. samm: Hotellibroneerimise tööriista loomine

Nii nagu tingimuslik töövoog, kuid nüüd püüab selle kinni vahendustarkvara!


In [4]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Samm 4: 🌟 Loo prioriteedikontrolli middleware (PEAMINE FUNKTSIOON!)

See on selle märkmiku **tuumafunktsionaalsus**. Middleware:

1. **Kasutab vahele** hotellibroneerimise funktsiooni kutse
2. **Täidab** funktsiooni tavapäraselt, kutsudes `next(context)`
3. **Kontrollib** tulemust `context.result`
4. **Üle kirjutab** tulemuse, kui kasutaja on prioriteet ja tube pole saadaval
5. **Tagastab** muudetud tulemuse agendile

**Oluline muster:**
```python
async def my_middleware(context, next):
    await next(context)  # Käivita funktsioon
    # Nüüd sisaldab context.result funktsiooni väljundit
    if some_condition:
        context.result = new_value  # Üle kirjutada!
```


In [5]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## Samm 5: Määra tingimusfunktsioonid marsruutimiseks

Sama tingimusfunktsioonid mis tingimuslikus töövoos - need kontrollivad struktureeritud väljundit marsruudi määramiseks.


In [6]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## Samm 6: Loo Kohandatud Kuvamise Versioniitja

Sama versioniitja mis varem - kuvab lõpliku töövoo väljundi.


In [7]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## Samm 7: Laadi keskkonnamuutujad

Seadista LLM klient (GitHub Models või OpenAI).


In [8]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## Samm 8: Loo tehisintellekti agentid vahemaagi abil

**PEAMINE ERINEVUS:** Kui loome availability_agent’i, anname me üle parameetri `middleware`!

Nii süstime me priority_check_middleware funktsiooni agenti kutse torujuhtmesse.


In [9]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Samm 9: Ehita töövoog

Sama töövoo struktuur nagu enne – tingimuslik marsruutimine vastavalt saadavusele.


In [10]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## Samm 10: Testjuhtum 1 - Tavaline kasutaja Pariisis (Ilma ülekatteta)

Tavaline kasutaja proovib broneerida Pariisi → Tube pole → Suunatakse alternative_agentile


In [11]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## Samm 11: Testjuhtum 2 - 🌟 Prioriteetne kasutaja Pariisis (ÜLEKIRJESTUSEGA!)

Prioriteetne liige proovib broneerida Pariis → Alguses pole tube → 🌟 Middleware kirjutab üle! → Suunab booking_agentile

**See on keskne demonstreerimine middleware'i võimsusest!**


In [12]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## Samm 12: Testjuht 3 - Prioriteetkasutaja Stockholmis (Juba saadaval)

Prioriteetkasutaja proovib Stockholmis → Ruume saadaval → Ülekirjutust ei ole vaja → Suunab booking_agenti juurde

See näitab, et middleware tegutseb ainult siis, kui seda vajatakse!


In [13]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## Peamised järeldused ja vahenduskihi kontseptsioonid

### ✅ Mida sa õppisid:

#### **1. Funktsioonipõhine vahenduskihi muster**

Vahenduskihi abil püütakse funktsioonikutsed kinni, kasutades lihtsat asünkroonset funktsiooni:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Enne funktsiooni täitmist
    print("Intercepting...")
    
    # Täida funktsioon
    await next(context)
    
    # Pärast funktsiooni täitmist - kontrolli tulemust
    if context.result:
        # Muuda tulemust vajadusel
        context.result = modified_value
```

#### **2. Konteksti juurde pääsemine ja tulemuse muutmine**

- `context.function` - Pääseb ligi kutsutavale funktsioonile
- `context.arguments` - Loeb funktsiooni parameetreid
- `context.kwargs` - Pääseb juurde täiendavatele parameetritele
- `await next(context)` - Käivitab funktsiooni
- `context.result` - Loeb/muudab funktsiooni väljundit

#### **3. Ärilogiika rakendamine**

Meie vahenduskihi kaudu rakendatakse prioriteetsete liikmete eeliseid:
- **Tavakasutajad**: muudatusi pole, standardne töövoog
- **Prioriteetsed kasutajad**: muudab "pole saadaval" → "saadaval"
- **Tingimuslik loogika**: muudatus ainult vajadusel

#### **4. Sama töövoog, erinevad tulemused**

Vahenduskihi võlu on:
- ✅ Töövoo struktuuri muudatusi pole
- ✅ Tööriista funktsiooni muudatusi pole
- ✅ Tingimusliku marsruudi loogikat ei muudeta
- ✅ Lihtsalt vahenduskihi kaudu → Erinev käitumine!

### 🚀 Reaalmaailma kasutusvõimalused:

1. **VIP/Premium funktsioonid**
   - Muudab tava limiite premium kasutajatele
   - Pakub prioriteetset juurdepääsu ressurssidele
   - Avab dünaamiliselt premium-funktsioone

2. **A/B testimine**
   - Suunab kasutajaid erinevatele rakendustele
   - Testib uusi funktsioone teatud kasutajatega
   - Funktsioonide järkjärguline juurutamine

3. **Turvalisus ja vastavus**
   - Kontrollib funktsioonikõnesid
   - Blokeerib tundlikud toimingud
   - Jõustab ärireegleid

4. **Jõudluse optimeerimine**
   - Vahemällu salvestab tulemused konkreetsetele kasutajatele
   - Jätab rasked toimingud vahele, kui võimalik
   - Dünaamiline ressursside jaotus

5. **Vigade käsitlemine ja taaskäivitamine**
   - Kinni ja käitleb vigu sujuvalt
   - Rakendab taaskäivitamise loogikat
   - Taganeb alternatiivsetele lahendustele

6. **Logimine ja monitooring**
   - Jälgib funktsioonide täitmise aegu
   - Logib parameetreid ja tulemusi
   - Monitoorib kasutusmustreid

### 🔑 Peamised erinevused dekoratsioonidest:

| Omadus | Dekoratsioon | Vahenduskihi |
|--------|--------------|--------------|
| **Ulatus** | Üks funktsioon | Kõik agenti funktsioonid |
| **Paindlikkus** | Kinnitatud definitsioonis | Dünaamiline jooksvalt |
| **Kontekst** | Piiratud | Täielik agendi kontekst |
| **Kompositsioon** | Mitu dekoratsiooni | Vahenduskihi torujuhe |
| **Agent-teadlik** | Ei | Jah (juurdepääs agendi olekule) |

### 📚 Millal kasutada vahenduskihte:

✅ **Kasuta vahenduskihte, kui:**
- Pead käitumist muutma kasutaja/seansi oleku põhjal
- Tahad rakendada loogikat mitmele funktsioonile
- Vajad ligipääsu agendi tasandi kontekstile
- Rakendad ristlõikavaid muresid (logimine, autentimine jne)

❌ **Ära kasuta vahenduskihte, kui:**
- Lihtne sisendi valideerimine (kasuta Pydanticut)
- Konkreetse funktsiooni loogika (hoia funktsioonis)
- Ühekordsed muudatused (muuda otse funktsiooni)

### 🎓 Täpsemad mustrid:

```python
# Mitme vahendustarkvara korral (tähtis on täitmise järjekord!)
middleware=[
    logging_middleware,      # Logib esmalt
    auth_middleware,         # Siis kontrollib autentimist
    cache_middleware,        # Siis kontrollib vahemälu
    rate_limit_middleware,   # Siis piirab sagedust
    priority_check_middleware  # Lõpuks prioriteedi kontroll
]

# Tingimuslik vahendustarkvara täitmine
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Muudab tulemust
    else:
        # Jätab täitmise täielikult vahele
        context.result = cached_value
```

### 🔗 Seotud kontseptsioonid:

- **Agendi vahenduskihi**: Püüab kinni agent.run() kutsed
- **Funktsiooni vahenduskihi**: Püüab kinni tööriistafunktsioonide kutsed (mida me kasutasime!)
- **Vahenduskihi torujuhe**: Järjestikune vahenduskihirida täitmises
- **Konteksti levitamine**: Edastab olekut vahenduskihi ahelas


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Vastutusest loobumine**:
See dokument on tõlgitud tehisintellekti tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palun arvestage, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Originaaldokument selle emakeeles tuleks pidada autoriteetseks allikaks. Tähtsa teabe puhul soovitame professionaalset inimtõlget. Me ei vastuta selles tõlkes sisalduvate eksimuste ega valesti mõistmiste eest.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
